In [1]:
from collections import namedtuple
import numpy as np
import math

In [2]:
Graph = namedtuple("Graph", ["nodes", "edges"])

nodes = ["A","B","C","D"]
edges = [
    ("A","B"),
    ("A","B"),
    ("A","C"),
    ("A","C"),
    ("A","D"),
    ("B","D"),
    ("C","D")
]

G = Graph(nodes,edges) 

adjacency_dict = {node:[] for node in G.nodes}

for edge in G.edges:
    node1, node2 = edge[0], edge[1]
    adjacency_dict[node1].append(node2)
    adjacency_dict[node2].append(node1)

adjacency_dict

{'A': ['B', 'B', 'C', 'C', 'D'],
 'B': ['A', 'A', 'D'],
 'C': ['A', 'A', 'D'],
 'D': ['A', 'B', 'C']}

In [3]:
[[5 for i in range(3)] for j in range(5)]

[[5, 5, 5], [5, 5, 5], [5, 5, 5], [5, 5, 5], [5, 5, 5]]

In [4]:
nodes = range(4)
edges = [
    (0,1),
    (0,1),
    (0,2),
    (0,2),
    (0,3),
    (1,3),
    (2,3)
]

G = Graph(nodes,edges)

adj = [[0 for node in G.nodes] for node in G.nodes] 

for edge in G.edges:
    node1,node2 = edge[0], edge[1]
    adj[node1][node2] += 1
    adj[node2][node1] += 1
adj

[[0, 2, 2, 1], [2, 0, 0, 1], [2, 0, 0, 1], [1, 1, 1, 0]]

<h1>Generating an adjacency matrix for a graph</h1>



<p>We define the graph nodes and edges as follows:</p>
<img src="graphs.png" height="400" >
<p> Then we generate a graph object using namedtuple from the collections module. </p>
<p>We then define the edges manually to then generate an adjacency matrix</p>

<h3>Define the graph</h3>

In [5]:
#define the graph
Graph = namedtuple("Graph",["nodes", "edges"]) #make a "class" and its "attributes"

nodes = range(7)
edges = [
    (0,1),
    (0,5),
    (0,6),
    (1,0),
    (1,2),
    (1,6),
    (2,1),
    (2,3),
    (2,6),
    (3,2),
    (3,4),
    (3,6),
    (4,3),
    (4,5),
    (4,6),
    (5,4),
    (5,6),
    (5,0),
    (6,1),
    (6,2),
    (6,3),
    (6,4),
    (6,5),
    (6,0)

]

G = Graph(nodes,edges)

In [6]:
numberofNodes = len(G.nodes)
matrix_shape = (numberofNodes, numberofNodes)
adj_matrix = np.zeros(matrix_shape)
for edge in G.edges:
    node1, node2 = edge[0], edge[1]
    adj_matrix[node1][node2] +=1
    adj_matrix[node2][node1] +=1

adj_matrix = adj_matrix/2 #just to make everything 1s and 0s



<h1>Creating transition matrices</h1>

      
<p>The following adjacency matrix was generated by the above code for the graph(see below):</p>
<img src="graphs.png" height="400" >
      
         0   1   2   3   4   5   6
      0 [0., 1., 0., 0., 0., 1., 1.]
      1 [1., 0., 1., 0., 0., 0., 1.]
      2 [0., 1., 0., 1., 0., 0., 1.]
      3 [0., 0., 1., 0., 1., 0., 1.]
      4 [0., 0., 0., 1., 0., 1., 1.]
      5 [1., 0., 0., 0., 1., 0., 1.]
      6 [1., 1., 1., 1., 1., 1., 0.]

<p>Using this adjacency matrix, we can easily create the transition matrix based off the probabilities defined in the figure.</p>
<p>The stay probability of a node is defined by the looping arrow.</p>
   <p>This stay probability is simply the edge connecting the node to itself. This is represented by the <strong>diagonal</strong> of the adjacency matrix.</p>
<p>The transition probability of a node (to the next node) is simply then : (1 - stay probability of that node) / number of adjacent nodes.</p>

In [7]:
def generate_transitionMatrix(graph,stay_probability=0):
    """Function to create the transition matrix. We assume that the graph has already been created and the 
    relevant nodes and edges are clearly defined. """

    #get the number of nodes
    numberofNodes = len(graph.nodes)
    #get shape of the transition matrix 
    matrix_shape = (numberofNodes, numberofNodes)
    #create the adjacent matrix with zeros
    adj_matrix = np.zeros(matrix_shape)

    for edge in graph.edges:
        node1,node2 = edge[0],edge[1]
        adj_matrix[node1][node2] += 1
        adj_matrix[node2][node1] += 1
    adj_matrix = adj_matrix/2 #just to make everything 1s and 0s
    
    #lets keep the calculation of the transition probabilites dynamic based off the number of adjacent nodes from
    #a starting node and the stay probability of that starting node
    #therefore what we should have is: transition probability = (1 - stay probability) / number of adjacent nodes

    np.fill_diagonal(adj_matrix,stay_probability) #set the stay probability in the adj matrix for the for loop 
    transition_matrix = np.copy(adj_matrix)

    #lets get the number of neighbours for each node by reading off the total number of 1s (adjacent nodes)
    for index,row in enumerate(adj_matrix):
        #we assume that the loops of a node to itself are already defined in the edges of the graph
        number_adjacent_nodes = np.count_nonzero(row == 1)
        transition_probability = (1 - stay_probability) / (number_adjacent_nodes)

        #adjust the decimal places
        transition_probability

        #replace the 1s (adjacent nodes) with their transition probability
        transition_matrix[index] = np.where(row ==1, f"{transition_probability:.2f}", row) #formatting the transition probability to two decimal places

    #set the stay probability of the matrix for each node 
    np.fill_diagonal(transition_matrix,stay_probability)
    #replace the diagonals with zeros for the adjacent matrix
    np.fill_diagonal(adj_matrix, 0)

    return adj_matrix, transition_matrix
        



In [8]:
#define graph, nodes and edges
#define the graph
Graph = namedtuple("Graph",["nodes", "edges"]) #make a "class" and its "attributes"

nodes = range(7)
edges = [
    (0,0), #loop back to itself
    (0,1),
    (0,5),
    (0,6),
    (1,1), #loop back to itself
    (1,0),
    (1,2),
    (1,6),
    (2,2), #loop back to itself
    (2,1),
    (2,3),
    (2,6),
    (3,3), #loop back to itself
    (3,2),
    (3,4),
    (3,6),
    (4,4), #loop back to itself
    (4,3),
    (4,5),
    (4,6),
    (5,5), #loop back to itself
    (5,4),
    (5,6),
    (5,0),
    (6,6), #loop back to itself
    (6,1),
    (6,2),
    (6,3),
    (6,4),
    (6,5),
    (6,0)

]

G = Graph(nodes,edges)

In [9]:
colour_adjmatrix, colour_transition_matrix = generate_transitionMatrix(G, stay_probability=0.1)
colour_adjmatrix, colour_transition_matrix

(array([[0., 1., 0., 0., 0., 1., 1.],
        [1., 0., 1., 0., 0., 0., 1.],
        [0., 1., 0., 1., 0., 0., 1.],
        [0., 0., 1., 0., 1., 0., 1.],
        [0., 0., 0., 1., 0., 1., 1.],
        [1., 0., 0., 0., 1., 0., 1.],
        [1., 1., 1., 1., 1., 1., 0.]]),
 array([[0.1 , 0.3 , 0.  , 0.  , 0.  , 0.3 , 0.3 ],
        [0.3 , 0.1 , 0.3 , 0.  , 0.  , 0.  , 0.3 ],
        [0.  , 0.3 , 0.1 , 0.3 , 0.  , 0.  , 0.3 ],
        [0.  , 0.  , 0.3 , 0.1 , 0.3 , 0.  , 0.3 ],
        [0.  , 0.  , 0.  , 0.3 , 0.1 , 0.3 , 0.3 ],
        [0.3 , 0.  , 0.  , 0.  , 0.3 , 0.1 , 0.3 ],
        [0.15, 0.15, 0.15, 0.15, 0.15, 0.15, 0.1 ]]))

In [10]:
texture_adjmatrix, texture_transition_matrix = generate_transitionMatrix(G, stay_probability=0.3)
texture_adjmatrix, texture_transition_matrix

(array([[0., 1., 0., 0., 0., 1., 1.],
        [1., 0., 1., 0., 0., 0., 1.],
        [0., 1., 0., 1., 0., 0., 1.],
        [0., 0., 1., 0., 1., 0., 1.],
        [0., 0., 0., 1., 0., 1., 1.],
        [1., 0., 0., 0., 1., 0., 1.],
        [1., 1., 1., 1., 1., 1., 0.]]),
 array([[0.3 , 0.23, 0.  , 0.  , 0.  , 0.23, 0.23],
        [0.23, 0.3 , 0.23, 0.  , 0.  , 0.  , 0.23],
        [0.  , 0.23, 0.3 , 0.23, 0.  , 0.  , 0.23],
        [0.  , 0.  , 0.23, 0.3 , 0.23, 0.  , 0.23],
        [0.  , 0.  , 0.  , 0.23, 0.3 , 0.23, 0.23],
        [0.23, 0.  , 0.  , 0.  , 0.23, 0.3 , 0.23],
        [0.12, 0.12, 0.12, 0.12, 0.12, 0.12, 0.3 ]]))

In [11]:
shape_adjmatrix, shape_transition_matrix = generate_transitionMatrix(G, stay_probability=0.5)
shape_adjmatrix, shape_transition_matrix

(array([[0., 1., 0., 0., 0., 1., 1.],
        [1., 0., 1., 0., 0., 0., 1.],
        [0., 1., 0., 1., 0., 0., 1.],
        [0., 0., 1., 0., 1., 0., 1.],
        [0., 0., 0., 1., 0., 1., 1.],
        [1., 0., 0., 0., 1., 0., 1.],
        [1., 1., 1., 1., 1., 1., 0.]]),
 array([[0.5 , 0.17, 0.  , 0.  , 0.  , 0.17, 0.17],
        [0.17, 0.5 , 0.17, 0.  , 0.  , 0.  , 0.17],
        [0.  , 0.17, 0.5 , 0.17, 0.  , 0.  , 0.17],
        [0.  , 0.  , 0.17, 0.5 , 0.17, 0.  , 0.17],
        [0.  , 0.  , 0.  , 0.17, 0.5 , 0.17, 0.17],
        [0.17, 0.  , 0.  , 0.  , 0.17, 0.5 , 0.17],
        [0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.5 ]]))

<h1>Creating Design Files - Pseudo Code</h1>
<p>Create an empty dataframe which should hold columns for</p>
    <ul>
    <li>Participant ID</li>
    <li>Block Number</li>
    <li>Trial Number</li>
    <li>Colour_stim</li>
    <li>Texture_stim</li>
    <li>Shape_stim</li>
    <li>Colour_choice</li>
    <li>Texture_choice</li>
    <li>Shape_stim</li>
    </ul>
<p>Set a variable for number of trials</p>
<p>Set a variable for number of participants for which we need to create design files</p>
<p>Set a flag for every 20th trial to trigger a choice trial.</p>
<p>Set up an array containing the three stay probabilities [0.1, 0.3, 0.5]. The stay probabilities need to be included as a parameter into the function to generate transition probabilities </p>
<p>Loop through the range of participants</p>
    <p>Pick a transition probability randomly for each of the dimensions. We do this to ensure participants  experience fast,medium and slow transitions of the dimensions </p>
    <p>Set up adjacency and transition matrices for each dimension using the functions with the chosen transition probs in the step above.</p>
    <p>Set up variables to hold current rows for colour, texture and shape</p>
    <p>Set up a variable for block, update this every 180 trials by +1. </p>
    <p>Loop through number of trials : </p>
        <ul>
        <li>Just for the first iteration, generate 3 random whole numbers between 1-7. These 3 numbers will be the index of the row of the transition matrix of each dimension. Record these 3 numbers as the current row</li>
        <li>These resulting numbers will be used to generate the first object (colour + texture + shape) </li>
        <li>From the second iteration onwards (for all the following shapes), generate 3 random numbers between 0 - 1.</li>
            <li>If the randomly generated number for all or any dimension, is greater than the stay probability of that dimension, we transition to the adjacent node. To transition, we do the following:</li>
            <li>To determine which node (so basically decide the next row) to transition to based off the current row, we look for adjacent nodes (columns) in the current row of the matrix where != 0.</li>
            <li>Note that sometimes the stay probabilities are greater than the transition probabilities.</li>
            <li> If the current row != 6 (7th row), then we follow the steps below to find the node to transition-</li>
            <li>If difference <= 0.4, transition to first neighbour, get column index, this number/index becomes current row. Update the respective dimension column(s) for that trial with the row number</li>
            <li>If difference >0.4 & <= 0.7, transition to second neighbour, get column index, this number/index becomes current row.</li>
            <li>If difference >0.7 , transition to third neighbour, get column index, this number/index becomes current row.</li>
            <li>If row == 6, then we randomly pick a neighbour to transition to.</li>
            <li>Every 20th trial, or rather If the trial%20 == 0, then we generate 3 random whole numbers for the choice columns for the choice trials  </li>
            <li>If the randomly generated number for all or any dimension, is less than stay probability, we don't transition. Therefore current row remains the same</li>
            <li>To include our choice trials, if trial % 20, we draw also draw 3 random numbers and add it to the colour_choice, texture_choice and shape_choice rows of the dataframe. Otherwise they remain NaNs</li>
            </ul>

In [12]:
import pandas as pd
import numpy as np
import random

In [13]:
#define graph, nodes and edges
#define the graph
Graph = namedtuple("Graph",["nodes", "edges"]) #make a "class" and its "attributes"

nodes = range(7)
edges = [
    (0,0), #loop back to itself
    (0,1),
    (0,5),
    (0,6),
    (1,1), #loop back to itself
    (1,0),
    (1,2),
    (1,6),
    (2,2), #loop back to itself
    (2,1),
    (2,3),
    (2,6),
    (3,3), #loop back to itself
    (3,2),
    (3,4),
    (3,6),
    (4,4), #loop back to itself
    (4,3),
    (4,5),
    (4,6),
    (5,5), #loop back to itself
    (5,4),
    (5,6),
    (5,0),
    (6,6), #loop back to itself
    (6,1),
    (6,2),
    (6,3),
    (6,4),
    (6,5),
    (6,0)

]

G = Graph(nodes,edges)

In [14]:
def generateChoiceTrials(total_trials, n_choice_trials, start_trial, jitter_range):
    """Function generates an array containing choice trials for the participants. The start trial parameter is used to
    to define the first choice trial. The total trials are of the experiment. The parameter
    n_choicetrials is the number of choice trials the participant gets. We define the starting trial and a jitter range.
    The minimum interval between trials are calculated inside the function and a random jitter from 0 to 5 is added
    to the interval. This makes the trials less predictable"""

    #generate the random choice trials between 1 and 240 trials with the first choice trial appearing at trial 10 or later

    #lets start with generating random trial numbers which are greater than 10 but less than 20
    #we start with a first trial being a number between 10 and 15
    # subsequent trials are generated if trial is greater than the previous trial and if the trial is between 10 and 20 increments from the previous trial
    #these trials must be a total of 18 trials and should be spaced out so they fill the span of 240 total trials
    minimum_interval = (total_trials - start_trial) // n_choice_trials
    choice_trials = []

    for i in range(n_choice_trials):
        base = start_trial + i * minimum_interval
        jitter = random.randint(0, jitter_range)
        subsequent_trial = min(base + jitter, total_trials - 1)
        choice_trials.append(subsequent_trial)

    return choice_trials

a = generateChoiceTrials(240*8,18*8,10,5)


In [15]:
   #could introduce a while loop here to ensure at least one of the dimensions of the randomly generated
                #numbers are not consistent with the graph structure for the choice trial

In [16]:
master_df = []

nTrials = 200
nParticipants = 5
nBlocks = 8
stayProbs = [0.1,0.3,0.5]
block = 1 #starting with just the first block
rowIndex = 0 #this is for when the inner loop trial finishes running for a block, the values for the next block start getting populated from the last index

for participant in range(nParticipants):
    
    #create dataframe for design file for every participant
    #regular columns
    columns = ["Participant_ID",
    "Block Number",
    "Trial Number",
    "Colour_stim",
    "Texture_stim",
    "Shape_stim",
    "Colour_choice",
    "Texture_choice",
    "Shape_choice",
    ]

    df = pd.DataFrame(columns=columns)

    #prepare the stayprobabilities and transition matrices for each dimension
    random.shuffle(stayProbs)

    colour_adjmatrix, colour_transition_matrix = generate_transitionMatrix(G,stay_probability=stayProbs[0])
    texture_adjmatrix, texture_transition_matrix = generate_transitionMatrix(G,stay_probability=stayProbs[1])
    shape_adjmatrix, shape_transition_matrix = generate_transitionMatrix(G,stay_probability=stayProbs[2])

    currentRow_colour = 0
    currentRow_texture = 0
    currentRow_shape = 0

    choiceTrialFlag = generateChoiceTrials(200*nBlocks,18*nBlocks,10,5)

    #assign the values to the columns
    for block in range(nBlocks):
        for trial in range(nTrials):
            df.loc[rowIndex,"Block Number"] = block + 1
            df.loc[rowIndex,"Trial Number"] = rowIndex + 1
            df.loc[rowIndex,"Stay_Probability(Colour)"] = stayProbs[0]
            df.loc[rowIndex,"Stay_Probability(Shape)"] = stayProbs[1]
            df.loc[rowIndex,"Stay_Probability(Texture)"] = stayProbs[2]

            if trial == 0: #for the very first trial generate 3 random numbers (integers)
                color_int = random.randint(0,6)
                texture_int = random.randint(0,6)
                shape_int = random.randint(0,6)

                df.loc[rowIndex, "Colour_stim"] = color_int
                df.loc[rowIndex, "Texture_stim"] = texture_int
                df.loc[rowIndex, "Shape_stim"] = shape_int

                currentRow_colour = color_int
                currentRow_texture = texture_int
                currentRow_shape = shape_int
            else:
                color_int = float(f"{random.uniform(0,1):.2f}")
                texture_int = float(f"{random.uniform(0,1):.2f}")
                shape_int = float(f"{random.uniform(0,1):.2f}")

                ##For the Colour dimension
                
                if color_int > stayProbs[0]: #check if the number is greater than the stay probability, to TRANSITION 
                    # print(f"Current node and the row in the transition matrix : {currentRow_colour}")
                    # print(f"random number is greater than stay probability, we can transition")

                    colour_node = colour_transition_matrix[currentRow_colour] #the row represents the node and all the values show the neighbours to which is connected to
                    #get the neighbouring nodes to which we can transition to (we exclude nodes that arent adjacent as well as the node to itself because we are transitioning )
                    adjacent_node_indices = np.where((colour_node != 0) & (colour_node != stayProbs[0]))
                    adj_node_transprobs = [colour_node[i] for i in adjacent_node_indices]
                    # print(f"All nodes in the row: {colour_node}, Stay probability : {stayProbs[0]}, Random Number Drawn:{color_int}, Adjacent nodes we can transition to{adjacent_node_indices[0]}, Number of adjacent nodes: {len(adjacent_node_indices[0])}, Transition Probabilities of those adjacent nodes : {adj_node_transprobs}")
                    thresholds = [0.4, 0.7, 1]
                    transition_node  = None

                    
                    if currentRow_colour == 6:
                        transition_node = random.choice(adjacent_node_indices[0])
                        # print(f"Current row == 6, transitioning to {transition_node}", "\n")
                        currentRow_colour = transition_node
                        df.loc[rowIndex, "Colour_stim"] = currentRow_colour
                    
                    else:
                        if color_int <= 0.4:
                            transition_node = adjacent_node_indices[0][0]
                            # print(f"Node to which are transitioning: {transition_node}","\n")
                        if (color_int > 0.4) and (color_int <= 0.7):
                            transition_node = adjacent_node_indices[0][1]
                            # print(f"Node to which are transitioning: {transition_node}","\n")
                        if (color_int > 0.7) and (color_int <= 1):
                            transition_node = adjacent_node_indices[0][2]
                            # print(f"Node to which are transitioning: {transition_node}","\n")
                    
                        currentRow_colour = transition_node
                        df.loc[rowIndex, "Colour_stim"] = currentRow_colour

                    

                else:
                    # print(f"Current node and the row in the transition matrix : {currentRow_colour}")
                    # print(f"random number: {color_int} is lower than stay probability {stayProbs[0]}, we stay", "\n")
                    df.loc[rowIndex, "Colour_stim"] = currentRow_colour

                ##For the texture dimension

                if texture_int > stayProbs[1]: #check if the number is greater than the stay probability, to TRANSITION 
                    # print(f"Current node and the row in the transition matrix : {currentRow_texture}")
                    # print(f"random number is greater than stay probability, we can transition")
                    texture_node = texture_transition_matrix[currentRow_texture] #the row represents the node and all the values show the neighbours to which is connected to
                    #get the neighbouring nodes to which we can transition to (we exclude nodes that arent adjacent as well as the node to itself because we are transitioning )
                    adjacent_node_indices = np.where((texture_node != 0) & (texture_node != stayProbs[1]))
                    adj_node_transprobs = [texture_node[i] for i in adjacent_node_indices]
                    # print(f"All nodes in the row: {texture_node}, Stay probability : {stayProbs[1]}, Random Number Drawn:{texture_int}, Adjacent nodes we can transition to{adjacent_node_indices[0]}, Number of adjacent nodes: {len(adjacent_node_indices[0])}, Transition Probabilities of those adjacent nodes : {adj_node_transprobs}")

                    thresholds = [0.4, 0.7, 1]
                    transition_node  = None

                    
                    if currentRow_texture == 6:
                        transition_node = random.choice(adjacent_node_indices[0])
                        # print(f"Current row == 6, transitioning to {transition_node}", "\n")
                        currentRow_texture = transition_node
                        df.loc[rowIndex, "Texture_stim"] = currentRow_texture
                    
                    else:
                        if texture_int <= 0.4:
                            transition_node = adjacent_node_indices[0][0]
                            # print(f"Node to which are transitioning: {transition_node}","\n")

                        if (texture_int > 0.4) and (texture_int <= 0.7):
                            transition_node = adjacent_node_indices[0][1]
                            # print(f"Node to which are transitioning: {transition_node}","\n")
                        if (texture_int > 0.7) and (texture_int <= 1):
                            transition_node = adjacent_node_indices[0][2]
                            # print(f"Node to which are transitioning: {transition_node}","\n")
                    
                        currentRow_texture = transition_node
                        df.loc[rowIndex, "Texture_stim"] = currentRow_texture

                else:
                    # print(f"Current node and the row in the transition matrix : {currentRow_texture}")
                    # print(f"random number: {texture_int} is lower than stay probability {stayProbs[1]}, we stay", "\n")
                    df.loc[rowIndex, "Texture_stim"] = currentRow_texture


                ##For the shape dimension
                
                if shape_int > stayProbs[2]: #check if the number is greater than the stay probability, to TRANSITION 
                    # print(f"Current node and the row in the transition matrix : {currentRow_shape}")
                    # print(f"random number is greater than stay probability, we can transition")
                    shape_node = shape_transition_matrix[currentRow_shape] #the row represents the node and all the values show the neighbours to which is connected to
                    #get the neighbouring nodes to which we can transition to (we exclude nodes that arent adjacent as well as the node to itself because we are transitioning )
                    adjacent_node_indices = np.where((shape_node != 0) & (shape_node != stayProbs[2]))
                    adj_node_transprobs = [shape_node[i] for i in adjacent_node_indices]
                    # print(f"All nodes in the row: {shape_node}, Stay probability : {stayProbs[2]}, Random Number Drawn:{shape_int}, Adjacent nodes we can transition to{adjacent_node_indices[0]}, Number of adjacent nodes: {len(adjacent_node_indices[0])}, Transition Probabilities of those adjacent nodes : {adj_node_transprobs}")

                    thresholds = [0.4, 0.7, 1]
                    transition_node  = None

                    
                    if currentRow_shape == 6:
                        transition_node = random.choice(adjacent_node_indices[0])
                        # print(f"Current row == 6, transitioning to {transition_node}", "\n")
                        currentRow_shape = transition_node
                        df.loc[rowIndex, "Shape_stim"] = currentRow_shape
                    
                    else:
                        if shape_int <= 0.4:
                            transition_node = adjacent_node_indices[0][0]
                            # print(f"Node to which are transitioning: {shape_node}","\n")

                        if (shape_int > 0.4) and (shape_int <= 0.7):
                            transition_node = adjacent_node_indices[0][1]
                            # print(f"Node to which are transitioning: {transition_node}","\n")
                        if (shape_int > 0.7) and (shape_int <= 1):
                            transition_node = adjacent_node_indices[0][2]
                            # print(f"Node to which are transitioning: {transition_node}","\n")
                    
                        currentRow_shape = transition_node
                        df.loc[rowIndex, "Shape_stim"] = currentRow_shape

                else:
                    # print(f"Current node and the row in the transition matrix : {currentRow_shape}")
                    # print(f"random number: {shape_int} is lower than stay probability {stayProbs[2]}, we stay", "\n")
                    df.loc[rowIndex, "Shape_stim"] = currentRow_shape

                if trial in choiceTrialFlag :
                    
                    color_int_choice = random.randint(0,6)
                    texture_int_choice = random.randint(0,6)
                    shape_int_choice = random.randint(0,6)

                    df.loc[rowIndex-1, "Colour_choice"] = color_int_choice
                    df.loc[rowIndex-1, "Texture_choice"] = texture_int_choice
                    df.loc[rowIndex-1, "Shape_choice"] = shape_int_choice

            rowIndex += 1

        df.iloc[:,0] = participant + 1
        #df.to_csv(f"../Experiment/DesignFiles/DesignFile_{participant+1}.csv")
                
            






C:\Users\Rishi\AppData\Local\Temp\ipykernel_14580\732118495.py:203: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.iloc[:,0] = participant + 1
C:\Users\Rishi\AppData\Local\Temp\ipykernel_14580\732118495.py:203: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df.iloc[:,0] = participant + 1
C:\Users\Rishi\AppData\Local\Temp\ipykernel_14580\732118495.py:203: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[

In [17]:
df

,Participant_ID,Block Number,Trial Number,Colour_stim,Texture_stim,Shape_stim,Colour_choice,Texture_choice,Shape_choice,Stay_Probability(Colour),Stay_Probability(Shape),Stay_Probability(Texture)
6400,5,1,6401,2,4,4,NaN,NaN,NaN,0.3,0.5,0.1
6401,5,1,6402,1,5,5,NaN,NaN,NaN,0.3,0.5,0.1
6402,5,1,6403,0,5,6,NaN,NaN,NaN,0.3,0.5,0.1
6403,5,1,6404,5,5,4,NaN,NaN,NaN,0.3,0.5,0.1
6404,5,1,6405,5,6,5,NaN,NaN,NaN,0.3,0.5,0.1
...,...,...,...,...,...,...,...,...,...,...,...,...
7995,5,8,7996,4,4,5,NaN,NaN,NaN,0.3,0.5,0.1
7996,5,8,7997,3,6,6,NaN,NaN,NaN,0.3,0.5,0.1
7997,5,8,7998,2,6,3,3,5,1,0.3,0.5,0.1
7998,5,8,7999,2,6,6,NaN,NaN,NaN,0.3,0.5,0.1


<h3>Creating a json file with the list of the folders in the design file folder</h3>

In [18]:
import os
import json

In [19]:
designFileList = os.listdir("../Experiment/DesignFiles")

with open("../Experiment/DesignFiles/DesignFileList.json", "w") as jsonFile:
    json.dump(designFileList, jsonFile)

